In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU,LSTM, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')

In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))  #查看时间长度为132

132


In [5]:
# 查看研究区域经纬度
ssh_lat = data_ssh['lat'][:].data
#print(ssh_lat[:])    #0.125 - 69.875
ssh_lon = data_ssh['lon'][:].data
#print(ssh_lon[:])    #110.125 - 259.875

In [6]:
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

(132, 280, 600)


In [7]:
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [9]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [10]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data
#print(sss_lon[:])

In [11]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')

In [14]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [15]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[440:1040])

# lat 0.125    ----    69.875
# lon  110.125    ----    259.875

In [16]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data

In [17]:
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST

In [19]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [20]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [21]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


In [22]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [23]:
sst = temp_depth[:,0:1,:,:]

In [24]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 3DS

In [25]:
data_3ds = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/sali/3DS_end_2005_2015.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [26]:
# 所有深度层
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]

#选取的深度层
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#[50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [27]:
sali_depth_use = sali_depth[:,1:2]

In [28]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        sali_depth_use = np.concatenate((sali_depth_use,sali_depth[:,i:i+1]),axis=1) 

In [29]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use,(0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


# 数据合并

In [30]:
# 在通道维度对数据进行合并
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [31]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)
print(cat_data_ss_depth.shape)

(132, 280, 600, 38)


## 将两个温度异常区域的数据设置为nan

In [32]:
# test = temp_depth_use[:,:,:,14:]
# test[test < 0.4] = np.nan
# test[test > 10] = np.nan
# cat_data_ss_depth[:,:,:,19:] = test

# 验证集和测试集的划分

In [33]:
# 最后一个月的数据作为test
test_num = 120
train = cat_data_ss_depth[:test_num]
test = cat_data_ss_depth[test_num:]
print('train:',train.shape,'test:',test.shape)

train: (120, 280, 600, 38) test: (12, 280, 600, 38)


## 去掉合并数据中的nan


In [34]:
train_flatten = train.reshape(-1,38)
#将默认填充值设置为nan
train_flatten[train_flatten == 32767] = np.nan
print('train删除nan值之前',train_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(train_flatten)
# 通过布尔索引删除NaN值
train_without_nan = train_flatten[~nan_mask.any(axis=1)]  #只要有一个是True就会返回True, 此处True代表nan，故求反
print('train删除nan值之后',train_without_nan.shape)

train删除nan值之前 (20160000, 38)
train删除nan值之后 (12567480, 38)


In [35]:
test_flatten =  test.reshape(-1,38)
test_flatten[test_flatten == 32767] = np.nan
print('test删除nan值之前',test_flatten.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(test_flatten)
# 通过布尔索引删除NaN值
test_without_nan = test_flatten[~nan_mask.any(axis=1)]
print('test删除nan值之后',test_without_nan.shape)

test删除nan值之前 (2016000, 38)
test删除nan值之后 (1256760, 38)


## 划分训练数据和目标数据

In [36]:
x_test = test_without_nan[:,:5]
y_test = test_without_nan[:,5:]

x_train_val = train_without_nan[:,:5]
y_train_val = train_without_nan[:,5:]

In [37]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train_val',x_train_val.shape)
print('y_train_val',y_train_val.shape)

x_test.shape: (1256760, 5)
y_test.shape (1256760, 33)
x_train_val (12567480, 5)
y_train_val (12567480, 33)


In [38]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=100)

In [39]:
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

((9425610, 5),
 (1256760, 5),
 (3141870, 5),
 (9425610, 33),
 (1256760, 33),
 (3141870, 33))

# 特征归一化

In [40]:
y_test_copy = y_test

In [41]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train) 
x_test = scaler_f.transform(x_test)
x_val = scaler_f.transform(x_val)

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train)
y_test = scaler_l.transform(y_test)
y_val = scaler_l.transform(y_val)

# 定义网络结构

In [42]:
# 创建神经网络模型
def create_model():   
    model = models.Sequential([
        layers.Dense(200, activation='tanh', input_shape=(5,)),  # 输入层和隐藏层
        layers.Dense(15, activation='tanh'),
        layers.Dense(8, activation='tanh'),
        layers.Dense(1, activation='linear')  # 输出层，num_depths表示深度的数量
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 
#model.summary()

## 网格搜索交叉验证寻找最佳参数

In [43]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [44]:
# param_grid = {
#     'neurons': [4,8,15,30,50,100,150,200],
# }

In [45]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=200,verbose=2,batch_size=1024)

In [46]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 编译模型

In [47]:

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                3015      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 4,352
Trainable params: 4,352
Non-trainable params: 0
_________________________________________________________________


In [48]:
# 创建神经网络模型
num_depths = 1
num_features = 5
# # 编译模型
# model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_path='./model/BPNN/sali/bpnn_model.h5'
keras_callbacks   = [
      EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
]

## 模型训练

### 对指定层进行训练

In [49]:
# num_epochs = 1000
# batch_size = 1024

# history = model.fit(sorted_data_x, sorted_data_y[:,14:], validation_data=(x_val,y_val[:,14:]) ,
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

### 对所有层进行训练

In [51]:
def BPNN_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [52]:
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33): #0-14
    BPNN_model(x_train, y_train[:,i:i+1],x_val, y_val[:,i:i+1],
               'D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/sali/model1/bpnn_sali_'+str(depth[i])+'m.h5')
 

Epoch 1/1000
921/921 - 3s - loss: 0.0092 - val_loss: 0.0026
Epoch 2/1000
921/921 - 1s - loss: 0.0025 - val_loss: 0.0024
Epoch 3/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 4/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 5/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 6/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 7/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 8/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 9/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 10/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 11/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 12/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 13/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 14/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0025
Epoch 15/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 16/1000
921/921 - 1s - loss: 0.0024 - val_loss: 0.0024
Epoch 17/1000
921/921 - 1s - loss

921/921 - 1s - loss: 0.0124 - val_loss: 0.0124
Epoch 13/1000
921/921 - 1s - loss: 0.0124 - val_loss: 0.0124
Epoch 14/1000
921/921 - 1s - loss: 0.0123 - val_loss: 0.0123
Epoch 15/1000
921/921 - 1s - loss: 0.0123 - val_loss: 0.0123
Epoch 16/1000
921/921 - 1s - loss: 0.0123 - val_loss: 0.0122
Epoch 17/1000
921/921 - 1s - loss: 0.0122 - val_loss: 0.0122
Epoch 18/1000
921/921 - 1s - loss: 0.0122 - val_loss: 0.0122
Epoch 19/1000
921/921 - 1s - loss: 0.0122 - val_loss: 0.0122
Epoch 20/1000
921/921 - 1s - loss: 0.0122 - val_loss: 0.0123
Epoch 21/1000
921/921 - 1s - loss: 0.0121 - val_loss: 0.0122
Epoch 22/1000
921/921 - 1s - loss: 0.0121 - val_loss: 0.0121
Epoch 23/1000
921/921 - 1s - loss: 0.0121 - val_loss: 0.0121
Epoch 24/1000
921/921 - 1s - loss: 0.0121 - val_loss: 0.0120
Epoch 25/1000
921/921 - 1s - loss: 0.0120 - val_loss: 0.0120
Epoch 26/1000
921/921 - 1s - loss: 0.0120 - val_loss: 0.0120
Epoch 27/1000
921/921 - 1s - loss: 0.0120 - val_loss: 0.0120
Epoch 28/1000
921/921 - 1s - loss: 0.0

Epoch 13/1000
921/921 - 1s - loss: 0.0228 - val_loss: 0.0227
Epoch 14/1000
921/921 - 1s - loss: 0.0227 - val_loss: 0.0226
Epoch 15/1000
921/921 - 1s - loss: 0.0226 - val_loss: 0.0225
Epoch 16/1000
921/921 - 1s - loss: 0.0225 - val_loss: 0.0224
Epoch 17/1000
921/921 - 1s - loss: 0.0224 - val_loss: 0.0224
Epoch 18/1000
921/921 - 1s - loss: 0.0223 - val_loss: 0.0223
Epoch 19/1000
921/921 - 1s - loss: 0.0222 - val_loss: 0.0222
Epoch 20/1000
921/921 - 1s - loss: 0.0222 - val_loss: 0.0221
Epoch 21/1000
921/921 - 1s - loss: 0.0221 - val_loss: 0.0221
Epoch 22/1000
921/921 - 1s - loss: 0.0220 - val_loss: 0.0221
Epoch 23/1000
921/921 - 1s - loss: 0.0220 - val_loss: 0.0219
Epoch 24/1000
921/921 - 1s - loss: 0.0219 - val_loss: 0.0219
Epoch 25/1000
921/921 - 1s - loss: 0.0219 - val_loss: 0.0219
Epoch 26/1000
921/921 - 1s - loss: 0.0219 - val_loss: 0.0218
Epoch 27/1000
921/921 - 1s - loss: 0.0218 - val_loss: 0.0219
Epoch 28/1000
921/921 - 1s - loss: 0.0218 - val_loss: 0.0218
Epoch 29/1000
921/921 - 

Epoch 40/1000
921/921 - 1s - loss: 0.0275 - val_loss: 0.0276
Epoch 41/1000
921/921 - 1s - loss: 0.0275 - val_loss: 0.0275
Epoch 42/1000
921/921 - 1s - loss: 0.0274 - val_loss: 0.0276
Epoch 43/1000
921/921 - 1s - loss: 0.0274 - val_loss: 0.0274
Epoch 44/1000
921/921 - 1s - loss: 0.0274 - val_loss: 0.0276
Epoch 45/1000
921/921 - 1s - loss: 0.0273 - val_loss: 0.0273
Epoch 46/1000
921/921 - 1s - loss: 0.0273 - val_loss: 0.0272
Epoch 47/1000
921/921 - 1s - loss: 0.0273 - val_loss: 0.0274
Epoch 48/1000
921/921 - 1s - loss: 0.0272 - val_loss: 0.0272
Epoch 49/1000
921/921 - 1s - loss: 0.0272 - val_loss: 0.0272
Epoch 50/1000
921/921 - 1s - loss: 0.0272 - val_loss: 0.0272
Epoch 51/1000
921/921 - 1s - loss: 0.0271 - val_loss: 0.0273
Epoch 52/1000
921/921 - 1s - loss: 0.0271 - val_loss: 0.0274
Epoch 53/1000
921/921 - 1s - loss: 0.0271 - val_loss: 0.0270
Epoch 54/1000
921/921 - 1s - loss: 0.0271 - val_loss: 0.0270
Epoch 55/1000
921/921 - 1s - loss: 0.0270 - val_loss: 0.0271
Epoch 56/1000
921/921 - 

Epoch 48/1000
921/921 - 1s - loss: 0.0332 - val_loss: 0.0332
Epoch 49/1000
921/921 - 1s - loss: 0.0332 - val_loss: 0.0332
Epoch 50/1000
921/921 - 1s - loss: 0.0332 - val_loss: 0.0332
Epoch 51/1000
921/921 - 1s - loss: 0.0331 - val_loss: 0.0333
Epoch 52/1000
921/921 - 1s - loss: 0.0331 - val_loss: 0.0331
Epoch 53/1000
921/921 - 1s - loss: 0.0331 - val_loss: 0.0331
Epoch 54/1000
921/921 - 1s - loss: 0.0331 - val_loss: 0.0331
Epoch 55/1000
921/921 - 1s - loss: 0.0330 - val_loss: 0.0330
Epoch 56/1000
921/921 - 1s - loss: 0.0330 - val_loss: 0.0330
Epoch 57/1000
921/921 - 1s - loss: 0.0330 - val_loss: 0.0330
Epoch 58/1000
921/921 - 1s - loss: 0.0330 - val_loss: 0.0329
Epoch 59/1000
921/921 - 1s - loss: 0.0329 - val_loss: 0.0331
Epoch 60/1000
921/921 - 1s - loss: 0.0329 - val_loss: 0.0331
Epoch 61/1000
921/921 - 1s - loss: 0.0329 - val_loss: 0.0328
Epoch 62/1000
921/921 - 1s - loss: 0.0329 - val_loss: 0.0329
Epoch 63/1000
921/921 - 1s - loss: 0.0328 - val_loss: 0.0328
Epoch 64/1000
921/921 - 

...

Epoch 194/1000
921/921 - 1s - loss: 0.2402 - val_loss: 0.2429
Epoch 195/1000
921/921 - 1s - loss: 0.2402 - val_loss: 0.2417
Epoch 196/1000
921/921 - 1s - loss: 0.2399 - val_loss: 0.2422
Epoch 197/1000
921/921 - 1s - loss: 0.2400 - val_loss: 0.2398
Epoch 198/1000
921/921 - 1s - loss: 0.2397 - val_loss: 0.2400
Epoch 199/1000
921/921 - 1s - loss: 0.2395 - val_loss: 0.2403
Epoch 200/1000
921/921 - 1s - loss: 0.2395 - val_loss: 0.2407
Epoch 201/1000
921/921 - 1s - loss: 0.2393 - val_loss: 0.2395
Epoch 202/1000
921/921 - 1s - loss: 0.2393 - val_loss: 0.2414
Epoch 203/1000
921/921 - 1s - loss: 0.2391 - val_loss: 0.2394
Epoch 204/1000
921/921 - 1s - loss: 0.2390 - val_loss: 0.2425
Epoch 205/1000
921/921 - 1s - loss: 0.2392 - val_loss: 0.2399
Epoch 206/1000
921/921 - 1s - loss: 0.2390 - val_loss: 0.2389
Epoch 207/1000
921/921 - 1s - loss: 0.2387 - val_loss: 0.2389
Epoch 208/1000
921/921 - 1s - loss: 0.2388 - val_loss: 0.2403
Epoch 209/1000
921/921 - 1s - loss: 0.2388 - val_loss: 0.2386
Epoch 21

921/921 - 1s - loss: 0.2289 - val_loss: 0.2289
Epoch 327/1000
921/921 - 1s - loss: 0.2288 - val_loss: 0.2305
Epoch 328/1000
921/921 - 1s - loss: 0.2287 - val_loss: 0.2295
Epoch 329/1000
921/921 - 1s - loss: 0.2286 - val_loss: 0.2282
Epoch 330/1000
921/921 - 1s - loss: 0.2286 - val_loss: 0.2295
Epoch 331/1000
921/921 - 1s - loss: 0.2284 - val_loss: 0.2293
Epoch 332/1000
921/921 - 1s - loss: 0.2282 - val_loss: 0.2306
Epoch 333/1000
921/921 - 1s - loss: 0.2283 - val_loss: 0.2290
Epoch 334/1000
921/921 - 1s - loss: 0.2283 - val_loss: 0.2292
Epoch 335/1000
921/921 - 1s - loss: 0.2281 - val_loss: 0.2290
Epoch 336/1000
921/921 - 1s - loss: 0.2281 - val_loss: 0.2324
Epoch 337/1000
921/921 - 1s - loss: 0.2283 - val_loss: 0.2312
Epoch 338/1000
921/921 - 1s - loss: 0.2280 - val_loss: 0.2297
Epoch 339/1000
921/921 - 1s - loss: 0.2278 - val_loss: 0.2287
Epoch 340/1000
921/921 - 1s - loss: 0.2279 - val_loss: 0.2333
Epoch 341/1000
921/921 - 1s - loss: 0.2279 - val_loss: 0.2316
Epoch 342/1000
921/921 

Epoch 4/1000
921/921 - 1s - loss: 0.4042 - val_loss: 0.3984
Epoch 5/1000
921/921 - 1s - loss: 0.3899 - val_loss: 0.3847
Epoch 6/1000
921/921 - 1s - loss: 0.3786 - val_loss: 0.3747
Epoch 7/1000
921/921 - 1s - loss: 0.3689 - val_loss: 0.3661
Epoch 8/1000
921/921 - 1s - loss: 0.3611 - val_loss: 0.3582
Epoch 9/1000
921/921 - 1s - loss: 0.3546 - val_loss: 0.3533
Epoch 10/1000
921/921 - 1s - loss: 0.3498 - val_loss: 0.3488
Epoch 11/1000
921/921 - 1s - loss: 0.3457 - val_loss: 0.3481
Epoch 12/1000
921/921 - 1s - loss: 0.3423 - val_loss: 0.3410
Epoch 13/1000
921/921 - 1s - loss: 0.3391 - val_loss: 0.3390
Epoch 14/1000
921/921 - 1s - loss: 0.3370 - val_loss: 0.3357
Epoch 15/1000
921/921 - 1s - loss: 0.3341 - val_loss: 0.3350
Epoch 16/1000
921/921 - 1s - loss: 0.3326 - val_loss: 0.3352
Epoch 17/1000
921/921 - 1s - loss: 0.3310 - val_loss: 0.3302
Epoch 18/1000
921/921 - 1s - loss: 0.3296 - val_loss: 0.3306
Epoch 19/1000
921/921 - 1s - loss: 0.3278 - val_loss: 0.3295
Epoch 20/1000
921/921 - 1s - l

Epoch 138/1000
921/921 - 1s - loss: 0.2851 - val_loss: 0.2858
Epoch 139/1000
921/921 - 1s - loss: 0.2849 - val_loss: 0.2852
Epoch 140/1000
921/921 - 1s - loss: 0.2847 - val_loss: 0.2834
Epoch 141/1000
921/921 - 1s - loss: 0.2844 - val_loss: 0.2837
Epoch 142/1000
921/921 - 1s - loss: 0.2842 - val_loss: 0.2841
Epoch 143/1000
921/921 - 1s - loss: 0.2840 - val_loss: 0.2867
Epoch 144/1000
921/921 - 1s - loss: 0.2839 - val_loss: 0.2827
Epoch 145/1000
921/921 - 1s - loss: 0.2837 - val_loss: 0.2839
Epoch 146/1000
921/921 - 1s - loss: 0.2832 - val_loss: 0.2814
Epoch 147/1000
921/921 - 1s - loss: 0.2833 - val_loss: 0.2847
Epoch 148/1000
921/921 - 1s - loss: 0.2829 - val_loss: 0.2816
Epoch 149/1000
921/921 - 1s - loss: 0.2831 - val_loss: 0.2819
Epoch 150/1000
921/921 - 1s - loss: 0.2827 - val_loss: 0.2833
Epoch 151/1000
921/921 - 1s - loss: 0.2825 - val_loss: 0.2812
Epoch 152/1000
921/921 - 1s - loss: 0.2822 - val_loss: 0.2820
Epoch 153/1000
921/921 - 1s - loss: 0.2822 - val_loss: 0.2818
Epoch 15

921/921 - 1s - loss: 0.2667 - val_loss: 0.2657
Epoch 271/1000
921/921 - 1s - loss: 0.2670 - val_loss: 0.2659
Epoch 272/1000
921/921 - 1s - loss: 0.2664 - val_loss: 0.2677
Epoch 273/1000
921/921 - 1s - loss: 0.2668 - val_loss: 0.2682
Epoch 274/1000
921/921 - 1s - loss: 0.2664 - val_loss: 0.2668
Epoch 275/1000
921/921 - 1s - loss: 0.2667 - val_loss: 0.2658
Epoch 276/1000
921/921 - 1s - loss: 0.2666 - val_loss: 0.2652
Epoch 277/1000
921/921 - 1s - loss: 0.2663 - val_loss: 0.2655
Epoch 278/1000
921/921 - 1s - loss: 0.2660 - val_loss: 0.2671
Epoch 279/1000
921/921 - 1s - loss: 0.2663 - val_loss: 0.2655
Epoch 280/1000
921/921 - 1s - loss: 0.2661 - val_loss: 0.2664
Epoch 281/1000
921/921 - 1s - loss: 0.2661 - val_loss: 0.2673
Epoch 282/1000
921/921 - 1s - loss: 0.2659 - val_loss: 0.2654
Epoch 283/1000
921/921 - 1s - loss: 0.2659 - val_loss: 0.2676
Epoch 284/1000
921/921 - 1s - loss: 0.2661 - val_loss: 0.2652
Epoch 285/1000
921/921 - 1s - loss: 0.2660 - val_loss: 0.2662
Epoch 286/1000
921/921 

## 模型评估

In [61]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)
    #print('%.3f' % rmse)
    
    #print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" %r2)  
    #print("%.4f" % r2)
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [56]:
# look_loss(history)

In [62]:
# 模型预测  
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
#      1  2  3 4  5  6  7  8  9  10 11 12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28   29   30   31    32   33   
result = [] 
for i in range(0,33):
    #best_model = tf.keras.models.load_model('./model/BPNN/sali/bpnn_model_'+str(depth[i])+'m_2.h5') 
    best_model = tf.keras.models.load_model('../model/new_0-2000m_24.6.6/BPNN/sali/model1/bpnn_sali_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m的温度')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

预测第5m的温度
Test RMSE: 0.053
R² score: 0.9972
预测第10m的温度
Test RMSE: 0.081
R² score: 0.9936
预测第15m的温度
Test RMSE: 0.086
R² score: 0.9928
预测第20m的温度
Test RMSE: 0.093
R² score: 0.9915
预测第30m的温度
Test RMSE: 0.118
R² score: 0.9863
预测第40m的温度
Test RMSE: 0.149
R² score: 0.9781
预测第50m的温度
Test RMSE: 0.175
R² score: 0.9700
预测第60m的温度
Test RMSE: 0.195
R² score: 0.9632
预测第70m的温度
Test RMSE: 0.204
R² score: 0.9602
预测第80m的温度
Test RMSE: 0.212
R² score: 0.9567
预测第90m的温度
Test RMSE: 0.216
R² score: 0.9542
预测第100m的温度
Test RMSE: 0.222
R² score: 0.9508
预测第125m的温度
Test RMSE: 0.250
R² score: 0.9361
预测第150m的温度
Test RMSE: 0.276
R² score: 0.9226
预测第175m的温度
Test RMSE: 0.297
R² score: 0.9107
预测第200m的温度
Test RMSE: 0.313
R² score: 0.9010
预测第225m的温度
Test RMSE: 0.309
R² score: 0.9045
预测第250m的温度
Test RMSE: 0.311
R² score: 0.9039
预测第275m的温度
Test RMSE: 0.316
R² score: 0.9014
预测第300m的温度
Test RMSE: 0.325
R² score: 0.8961
预测第350m的温度
Test RMSE: 0.344
R² score: 0.8841
预测第400m的温度
Test RMSE: 0.353
R² score: 0.8786
预测第500m的温度
Test RMSE: 

# 预测结果可视化

## 反归一化

In [63]:
# 查看预测结果的形状
len(result),len(result[0]),len(result[0][0])

(33, 1256760, 1)

In [64]:
# 结果备份
result_copy = result

In [65]:
# 改变形状  (33, 1256760, 1) =》(1256760, 33)
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

(33, 1256760, 1)
(1256760, 1, 33)
(1256760, 33)


In [70]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)

In [71]:
result_unscaled.shape,y_test.shape

((1256760, 33), (1256760, 33))

In [72]:
# 查看反归一化之后的RMSE
for i in range(0,33):
    print('预测第'+str(depth[i])+'m的盐度')
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('Test RMSE: %.3f' % rmse)

预测第5m的盐度
Test RMSE: 0.045
预测第10m的盐度
Test RMSE: 0.068
预测第15m的盐度
Test RMSE: 0.072
预测第20m的盐度
Test RMSE: 0.078
预测第30m的盐度
Test RMSE: 0.097
预测第40m的盐度
Test RMSE: 0.122
预测第50m的盐度
Test RMSE: 0.142
预测第60m的盐度
Test RMSE: 0.157
预测第70m的盐度
Test RMSE: 0.162
预测第80m的盐度
Test RMSE: 0.167
预测第90m的盐度
Test RMSE: 0.167
预测第100m的盐度
Test RMSE: 0.167
预测第125m的盐度
Test RMSE: 0.169
预测第150m的盐度
Test RMSE: 0.163
预测第175m的盐度
Test RMSE: 0.154
预测第200m的盐度
Test RMSE: 0.143
预测第225m的盐度
Test RMSE: 0.126
预测第250m的盐度
Test RMSE: 0.117
预测第275m的盐度
Test RMSE: 0.111
预测第300m的盐度
Test RMSE: 0.108
预测第350m的盐度
Test RMSE: 0.103
预测第400m的盐度
Test RMSE: 0.097
预测第500m的盐度
Test RMSE: 0.082
预测第600m的盐度
Test RMSE: 0.070
预测第700m的盐度
Test RMSE: 0.063
预测第800m的盐度
Test RMSE: 0.054
预测第900m的盐度
Test RMSE: 0.046
预测第1000m的盐度
Test RMSE: 0.041
预测第1100m的盐度
Test RMSE: 0.037
预测第1300m的盐度
Test RMSE: 0.035
预测第1500m的盐度
Test RMSE: 0.036
预测第1750m的盐度
Test RMSE: 0.037
预测第2000m的盐度
Test RMSE: 0.041


## 为预测值填充nan

In [73]:
test_flatten.shape

(2016000, 38)

In [74]:
nan_mask = np.isnan(test_flatten)
nan_masks = nan_mask.any(axis=1) # true 代表通道中有nan   
print(nan_masks.shape)

(2016000,)


In [77]:
### 创建一个形状与nan_masks相同且全是nan的数组
result_with_nan = np.full(((nan_masks.shape[0]),33), np.nan)  

In [78]:
result_with_nan.shape,result.shape 

((2016000, 33), (1256760, 33))

In [79]:
np.count_nonzero(nan_masks)

759240

In [82]:
num = 0
#for depth in range(0,15):
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        result_with_nan[i] = result_unscaled[num]
        num = num + 1
        
print(num)

1256760


## 为归一化之后的真实值填充nan 

In [83]:
y_test_with_nan = np.full(((nan_masks.shape[0]),33), np.nan) 

In [84]:
y_test_with_nan.shape,y_test.shape 

((2016000, 33), (1256760, 33))

In [85]:
num = 0 
for i,nan_mask in enumerate(nan_masks):   # i从0开始
    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        y_test_with_nan[i] = y_test_unscaled[num]
        num = num + 1     
print(num)

1256760


# 结果可视化

In [87]:
result_with_nan.shape,y_test_with_nan.shape

((2016000, 33), (2016000, 33))

In [88]:
result_with_nan_reshape = result_with_nan.reshape((12,280,600,33))
y_test_with_nan_reshape = y_test_with_nan.reshape((12,280,600,33))
print(result_with_nan_reshape.shape , y_test_with_nan_reshape.shape)

(12, 280, 600, 33) (12, 280, 600, 33)


In [89]:
result_with_nan_reshape = np.transpose(result_with_nan_reshape,(0,3,1,2))  
y_test_with_nan_reshape = np.transpose(y_test_with_nan_reshape,(0,3,1,2))

print(result_with_nan_reshape.shape,y_test_with_nan_reshape.shape)

(12, 33, 280, 600) (12, 33, 280, 600)


In [90]:
# 获取经纬度和时间

lat = []
lon = []
time = []
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)


start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [91]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/BPNN/sali/result1/result5-2000m_sali.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('true_oss', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_oss', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months'
time_var.axis = 'T'

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
#new_NC.variables['level'][:] = level

new_NC.variables['true_oss'][:]=np.array(y_test_with_nan_reshape)
new_NC.variables['pred_oss'][:]=np.array(result_with_nan_reshape)

#最后记得关闭文件
new_NC.close()